In [ ]:
from xgeo import regrid, io, utils
import xarray as xr
import rasterio
import pandas as pd
from datetime import datetime
import numpy as np
from os.path import join

## get forcing for w3ra model

In [ ]:
start_date = datetime(2000, 1, 1)
end_date = datetime(2000, 12, 31)
bounds = (7.0, 48.0, 17.0, 55.0)
chunks={'lat': 100, 'lon': 100}
t_m = pd.date_range(start_date, end_date, freq='MS')
t_y = pd.date_range(start_date, end_date, freq='AS')

In [ ]:
modeldir = r'/home/dirk/experiments/model_test_data/test_Elbe/WFL_Elbe/W3RA'
like = join(modeldir, r'clone_elbe_30min.map')
reproject_kwargs = dict(like=like, resampling='average')

In [ ]:
rootdir = r'/home/dirk/datasets/E2O'
precip_fn = join(rootdir, 'mswep', '{year}.nc')
tair_fn = join(rootdir, 'met_forcing_v1', 'Tair_daily_EI_025_{year}{month}.nc')
psurf_fn = join(rootdir, 'met_forcing_v1', 'PSurf_daily_EI_025_{year}{month}.nc')
wind_fn = join(rootdir, 'met_forcing_v1', 'Wind_daily_EI_025_{year}{month}.nc')
pet_fn = join(rootdir, 'deltares', 'PeHar_daily_wrr2_0083_{year}.nc')

In [ ]:
tair_fns = [tair_fn.format(year='{:04d}'.format(t.year), month='{:02d}'.format(t.month)) for t in t_m]
psurf_fns = [psurf_fn.format(year='{:04d}'.format(t.year), month='{:02d}'.format(t.month)) for t in t_m]
wind_fns = [wind_fn.format(year='{:04d}'.format(t.year), month='{:02d}'.format(t.month)) for t in t_m]
pet_fns = [pet_fn.format(year='{:04d}'.format(t.year)) for t in t_y]
precip_fns = [precip_fn.format(year='{:04d}'.format(t.year)) for t in t_y]

In [ ]:
def process_da(da, start_date, end_date, bounds, reproject_kwargs):
    """clip, regrid and set crs metadata for forcing datasets"""
    # select / clip
    da = da.sel(**{'time': slice(start_date, end_date)})
    da = regrid.clip(da, bounds)
    # set metadata
    da = utils.set_crs(da, crs=rasterio.crs.CRS.from_epsg(4326))
    # regrid
    da = regrid.reproject_rasterio(da, **reproject_kwargs)
    return da

In [ ]:
psurf = xr.open_mfdataset(psurf_fns, chunks=chunks)['PSurf']
psurf = process_da(psurf, start_date, end_date, bounds, reproject_kwargs)

In [ ]:
wind = xr.open_mfdataset(wind_fns[:1], chunks=chunks)['Wind']
wind = wind.drop('height').squeeze()
wind = process_da(wind, start_date, end_date, bounds, reproject_kwargs)

In [ ]:
precip = xr.open_mfdataset(precip_fns, chunks=chunks)['precipitation']
precip = process_da(precip, start_date, end_date, bounds, reproject_kwargs)

In [ ]:
pet = xr.open_mfdataset(pet_fns, chunks=chunks)['PET']
res=1/12.
pet['lat'].data = np.arange(90-res/2., -90, -res)
pet['lon'].data = np.arange(-180+res/2., 180, res)
pet = process_da(pet, start_date, end_date, bounds, reproject_kwargs)

In [ ]:
tair = xr.open_mfdataset(tair_fns, chunks=chunks)['Tair']
tair = tair.drop('height').squeeze()
tair = process_da(tair, start_date, end_date, bounds, reproject_kwargs)

rename forcing data and save to netcdf

In [ ]:
tair.name = 'TDAY'
pet.name = 'EPOT'
precip.name = 'PRECIP'
wind.name = 'WIND'
psurf.name = 'PRESS'
ds_out = xr.merge([precip, pet, tair, wind, psurf])
ds_out.to_netcdf(join(modeldir, 'inmaps', 'e2o_wrr2_forcing_30min_elbe_2000.nc'))

save outputs to PCRaster mapstack

In [ ]:
# io.to_mapstack(tair, out_dir=join(modeldir, 'inmaps'), driver='PCRaster')
# io.to_mapstack(precip.astype('float32'), out_dir=join(modeldir, 'inmaps'), driver='PCRaster')
# io.to_mapstack(pet, out_dir=join(modeldir, 'inmaps'), driver='PCRaster')
# io.to_mapstack(tair, out_dir=join(modeldir, 'inmaps'), driver='PCRaster')
# io.to_mapstack(wind, out_dir=join(modeldir, 'inmaps'), driver='PCRaster')
# io.to_mapstack(psurf, out_dir=join(modeldir, 'inmaps'), driver='PCRaster')

## clip static maps and state maps from global to local models

In [ ]:
import rasterio 
from rasterio.windows import Window
import numpy as np
import os 
from os.path import join, basename, dirname
import glob
modeldir_glob = r'/home/dirk/experiments/wflow_cases/w3ra/openstreams_w3ra'
pcr_vs = {'u': 'VS_BOOLEAN', 'b': 'VS_BOOLEAN', 'f': 'VS_SCALAR', 'i': 'VS_NOMINAL'}
folder = 'staticmaps' #'instate'

with rasterio.open(like) as template_ds:
    bounds = template_ds.bounds

for fn in glob.glob(join(modeldir_glob, 'staticmaps', '*.map')):
    out_fn = join(modeldir, 'staticmaps', basename(fn)) 
    if os.path.isfile(out_fn):
        continue
    print(out_fn)
    with rasterio.open(fn, driver='PCRaster') as src:
        bounds_window = src.window(*bounds)
        bounds_window = bounds_window.intersection(
            Window(0, 0, src.width, src.height))
        out_window = bounds_window.round_lengths(op='ceil')
        height = int(out_window.height)
        width = int(out_window.width)
        
        out_kwargs = src.profile
        out_kwargs.update({
            'height': height,
            'width': width,
            'transform': src.window_transform(out_window)})
        out_kwargs.update(PCRASTER_VALUESCALE=pcr_vs.get(np.dtype(src.dtypes[0]).kind))
    
        with rasterio.open(out_fn, 'w', **out_kwargs) as out:
            out.write(src.read(window=out_window,
                               out_shape=(src.count, height, width)))

In [ ]:
# modeldir = r'/home/dirk/experiments/wflow_cases/w3ra/openstreams_w3ra'
# set lower case for all maps (required on linux!)
import os 
from os.path import join, basename, dirname
import glob
for fn in glob.glob(join(modeldir, 'staticmaps', '*.map')):
    os.rename(fn, join(dirname(fn), basename(fn).lower()))

In [ ]:
# modeldir = r'/home/dirk/experiments/wflow_cases/w3ra/openstreams_w3ra'
import os 
import shutil
from os.path import join, basename, dirname
import glob
for fn in glob.glob(join(modeldir, 'instate', '*.map')):
    shutil.copy(fn, join(dirname(fn), basename(fn).lower())) #.replace('.map', '_0.map')))